In [1]:
import wntr  
import numpy as np 
import pandas as pd
import re
import pathlib
import iws_modelling

In [4]:
 # Replace with appropriate path and filename
directory=pathlib.Path("../Network-Files/Network 1")
filename=pathlib.Path("Network2_4hr_PDA.inp")
name_only=filename.stem.split("_")[0:-2]
name_only="_".join(name_only)+"_"
print("Selected File: ",name_only)
path=directory/filename
path.with_stem(path.stem[0:-4])

Selected File:  Network2_


WindowsPath('../Network-Files/Network 1/Network2_4hr.inp')

In [5]:
iws_modelling.to_all(directory/pathlib.Path("Network1_8hr_PDA.inp"),0,10,100)

Selected File:  Network1_8hr_PDA
Selected File:  Network1_8hr_PDA
..\Network-Files\Network 1\Network1_8hr_CV-Tank.inp
Selected File:  Network1_8hr_PDA
Selected File:  Network1_8hr_PDA
Selected File:  Network1_8hr_PDA
Selected File:  Network1_8hr_PDA
Selected File:  Network1_8hr


[WindowsPath('../Network-Files/Network 1/Network1_8hr_CV-Res.inp'),
 WindowsPath('../Network-Files/Network 1/Network1_8hr_CV-Tank.inp'),
 WindowsPath('../Network-Files/Network 1/Network1_8hr_FCV-EM.inp'),
 WindowsPath('../Network-Files/Network 1/Network1_8hr_FCV-Res.inp'),
 WindowsPath('../Network-Files/Network 1/Network1_8hr_PSV-Tank.inp'),
 WindowsPath('../Network-Files/Network 1/Network1_8hr_100m_Outlet-Outfall.inp'),
 WindowsPath('../Network-Files/Network 1/Network1_8hr_100m_Outlet-Storage.inp')]

In [9]:
demand_nodes=[]       # For storing list of nodes that have non-zero demands
desired_demands=[]    # For storing demand rates desired by each node for desired volume calculations
elevations=[]

# Creates a network model object using EPANET .inp file
network=wntr.network.WaterNetworkModel(path)

# Iterates over the junction list in the Network object
for node in network.junctions():
    # Record node ID (name), desired demand (base_demand) in CMS, elevations, x and y coordinates
    demand_nodes.append(node[1].name)
    desired_demands.append(node[1].base_demand)
    elevations.append(node[1].elevation)


# Get the supply duration in minutes (/60) as an integer
supply_duration=int(network.options.time.duration/60)

In [10]:
new_duration_hr=8
new_duration_min=0
new_duration=new_duration_hr*60+new_duration_min
demand_multiplier=supply_duration/new_duration
if new_duration_min <10:
    new_duration_min="0"+str(new_duration_min)
else: new_duration_min=str(new_duration_min)

In [11]:
desired_demands=[demand*demand_multiplier*1000 for demand in desired_demands]
patterns=["       " for demand in desired_demands]
semicolons=[";" for demand in desired_demands]
node_section=pd.DataFrame(list(zip(demand_nodes,elevations,desired_demands,patterns,semicolons)))
node_section=node_section.to_string(header=False,index=False,col_space=10).splitlines()

In [12]:
# opens .inp file to read
file=open(path,'r')
lines=[]            # list to store all lines in the .inp file
linecount=0         # Counter for the number of lines
junctions_marker=0  # To store the line number at which the junctions section starts
supply_duration_line=0

# Loops over each line in the input file 
for line in file:
    # Record the position of the phrase [JUNCTIONS] and add 2 to skip the header line
    if re.search('\[JUNCTIONS\]',line):
        junctions_marker=linecount+2
    if re.search('Duration',line):
        supply_duration_line=linecount
    linecount+=1
    # Store all lines in a list
    lines.append(line)
file.close()



# Inserts the created sections in their appropriate location in the list of lines
lines[supply_duration_line]="Duration      "+str(new_duration_hr)+":"+new_duration_min+"\n"
lines[junctions_marker:junctions_marker+len(node_section)]=node_section

print(lines[supply_duration_line])
# Opens a new file in the same directory to write the modified network .inp file in
file=open(path.parent/pathlib.Path(name_only+str(new_duration_hr)+"hr"+'_PDA.inp'),'w')
c=0     #line counter

# All lines added by this script are missing a new line character at the end, the conditional statements below add the new line character for these lines only and writes all lines to the file
for line in lines:
    if c>=junctions_marker and c<=junctions_marker+len(node_section)+len(node_section):
        file.write(line+'\n')
    else: file.write(line)    
    c+=1
file.close()

Duration      8:00

